# Deezeer interaction

In [1]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
USER_ID = "1845554682" #ID of Nathan's Deezer account

In [2]:
from deezerus import create_dataframe

create_dataframe(USER_ID, full_version=True)

2025-10-08 19:00:53,166 - INFO - NumExpr defaulting to 8 threads.
2025-10-08 19:01:24,145 - INFO - Found 39 playlists for user Nathan
2025-10-08 19:01:24,146 - INFO - Processing 39 playlists...
2025-10-08 19:01:31,459 - INFO - Playlist '100' contains 0 tracks.
2025-10-08 19:01:31,910 - INFO - Playlist '?XD' contains 62 tracks.
2025-10-08 19:02:02,391 - ERROR - Error processing track ID 398055682: HTTPSConnectionPool(host='api.deezer.com', port=443): Max retries exceeded with url: /track/398055682 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025441CECFA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
2025-10-08 19:02:13,495 - ERROR - Error processing track ID 472992612: HTTPSConnectionPool(host='api.deezer.com', port=443): Max retries exceeded with url: /track/472992612 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000025441CEC5E0>: Failed to establish a new connection: [Errno 11001] get

,id,title,artist,album,duration,rank,release_date,bpm,gain,?XD,...,POJ,POK,RCN,RFR,RUS,SGM,SLV,SMR,SNC,VCL
0,63510361,Demons,Imagine Dragons,Night Visions,175,959778,2013-02-05,89.88,-6.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,135190564,Summer Time Record,JIN,Mekakucity M's 2 ～Mekakucity Actors Vocal & So...,278,86130,2015-06-03,178.21,-7.4,True,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2117010197,Voices (feat. Oliver Tree),KSI,Voices (feat. Oliver Tree),188,338484,2023-01-27,0.0,-7.3,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1619949302,Top Of The World,Andrew Michael Britton,Hard Knocks,183,25903,2021-07-15,0.0,-9.1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1710725517,UH OH!,Sub Urban,UH OH!,133,449851,2022-04-22,0.0,-10.4,True,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,1190172582,タイムマシン feat.GUMI,164,THIS IS VOCAROCK feat.GUMI,248,39565,2014-07-16,0,-7.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2246,1190172492,sleeping beauty feat.GUMI,164,THIS IS VOCAROCK feat.GUMI,236,96708,2014-07-16,0,-7.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2247,668973232,Love Trial,40mP,41m,219,177172,2014-08-17,0,-6.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2248,1085705272,Shamer,Chinozo,シェーマ,207,127568,2020-09-20,0,-9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
from tqdm import tqdm
import pandas as pd

track = None
df_tracks = pd.DataFrame(columns=["id", "title", "album", "duration", "release_date", "rank", "bpm", "gain"])

for playlist_id in PLAYLIST_ID_LIST:
    playlist = fetch_with_retry(f"https://api.deezer.com/playlist/{playlist_id}")
    playlist_name = f'in_{playlist["title"].replace(" ", "_")}'

    track_id_list = [playlist["tracks"]["data"][i]["id"] for i in range(len(playlist["tracks"]["data"]))]

    for track_id in track_id_list:
        if track_id not in df_tracks["id"].values:
            try:
                track = fetch_with_retry(f"https://api.deezer.com/track/{track_id}")
                artists = ([artist["name"] for artist in track["contributors"]])
                df_tracks = pd.concat([df_tracks, pd.DataFrame([{
                    "id": track["id"],
                    "title": track["title"],
                    "album": track["album"]["title"],
                    "duration": track["duration"],
                    "release_date": track["album"]["release_date"],
                    "rank": track["rank"],
                    "bpm": track["bpm"],
                    "gain": track["gain"],
                    f"{playlist_name}": True
                }])], ignore_index=True)
                for i in range(len(artists)):
                    df_tracks.loc[df_tracks.index[-1], f'artist_{i+1}'] = artists[i] if i < len(artists) else None

            except Exception as e:
                print(f"Failed to fetch track {track_id}: {e}")

        else:
            df_tracks.loc[df_tracks["id"] == track_id, f'{playlist_name}'] = True


artist_columns = [col for col in df_tracks.columns if col.startswith("artist")]
in_columns = [col for col in df_tracks.columns if col.startswith("in_")]
other_columns = [col for col in df_tracks.columns if col not in artist_columns and col not in in_columns]
df_tracks = df_tracks[other_columns + artist_columns + in_columns]

df_tracks.head()

C:\Users\natha\AppData\Local\Temp\ipykernel_26828\4030964714.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_tracks = pd.concat([df_tracks, pd.DataFrame([{


Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...
Quota exceeded. Retry 1/5 in 2s...


,id,title,album,duration,release_date,rank,bpm,gain,artist_1,artist_2,...,in_CUD,in_ONE,in_ADD,in_RCN,in_SGM,in_VCL,in_LRB,in_FAV,in_JFA,in_EFA
0,347419311,Ai Wo Tsutaetaidatoka,Ai wo Tsutaetaidatoka,235,2017-05-03,310792,99.86,-7.5,aimyon,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2274138747,ai no hana,ai no hana,241,2023-06-07,256356,0.00,-7.1,aimyon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1425383492,wi(l)d-screen baroque,Gekijouban Shojokageki Revuestarlight Gekichuk...,260,2021-07-21,110931,0.00,-8.7,Daiba Nana(CV:Moeka Koizumi),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN
3,135189966,Daze（Re Ver.）,Mekakucity M's 1 ～Mekakucity Actors Vocal & So...,233,2015-04-01,166410,197.82,-6.0,JIN,Maria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN
4,135189976,Kagerou Daze,Mekakucity M's 1 ～Mekakucity Actors Vocal & So...,243,2015-04-01,114657,192.30,-6.9,JIN,Taguchi Shoichi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
duplicate_titles = df_tracks[df_tracks.duplicated(subset='title', keep=False)]
duplicate_titles

,id,title,album,duration,release_date,rank,bpm,gain,artist_1,artist_2,...,in_CUD,in_ONE,in_ADD,in_RCN,in_SGM,in_VCL,in_LRB,in_FAV,in_JFA,in_EFA
56,1400300352,Phoenix,TOKYO,248,2021-06-23,400378,0.00,-7.6,BURNOUT SYNDROMES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2205268347,WORK,WORK,201,2023-04-01,21735,0.00,-9.2,ꉈꀧ꒒꒒ꁄꍈꍈꀧ꒦ꉈ ꉣꅔꎡꅔꁕꁄ,Sheena Ringo,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,1100228952,Flamingo,STRAY SHEEP,196,2020-08-05,295767,0.00,-9.1,Kenshi Yonezu,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN
107,118072614,Flamingo,Flamingo,197,2014-09-23,59805,178.21,-7.5,Kero Kero Bonito,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN
196,1228437032,Nightmare,HOPE,223,2021-02-17,17958,0.00,-6.4,Seven Billion Dots,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,1914090367,UNSTOPPABLE,R·I·O·T,230,2018-12-12,25364,0.00,-6.2,RAISE A SUILEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,1786490627,UNSTOPPABLE,ERA,230,2020-08-19,104284,0.00,-6.2,RAISE A SUILEN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,1228409662,Crazy,PURE,190,2018-11-28,11678,0.00,-9.0,Taichi Mukai,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,1100435222,STAY,eyes,204,2020-05-27,88683,0.00,-8.5,milet,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,593087512,The Greatest Show,The Greatest Showman: Reimagined (Deluxe),302,2017-12-08,165528,157.80,-8.6,Hugh Jackman,Keala Settle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
all_artists = df_tracks[artist_columns].stack().reset_index(level=1, drop=True).to_frame('artist')
all_artists['in_FAV'] = df_tracks.loc[all_artists.index, 'in_FAV'].values

artist_counts = all_artists['artist'].value_counts().to_frame('nombre_de_titres')
fav_counts = all_artists[all_artists['in_FAV'] == True]['artist'].value_counts().to_frame('nombre_de_favoris')

artist_summary = artist_counts.join(fav_counts, how='left').fillna(0)
artist_summary['coef'] = round((artist_summary['nombre_de_titres'] * 0.1) + (artist_summary['nombre_de_favoris'] * 0.25), 3)
artist_summary.reset_index(inplace=True)
artist_summary.rename(columns={'index': 'artiste'}, inplace=True)
artist_summary.sort_values(by='coef', ascending=False, inplace=True)
artist_summary

,artist,nombre_de_titres,nombre_de_favoris,coef
0,milet,44,13.0,7.65
1,League of Legends,24,9.0,4.65
2,MIYAVI,24,7.0,4.15
4,Shayfer James,14,6.0,2.90
5,NateWantsToBattle,14,5.0,2.65
...,...,...,...,...
627,Sheet Music Boss,1,0.0,0.10
626,Ari Pulkkinen,1,0.0,0.10
625,C418,1,0.0,0.10
624,Robert White,1,0.0,0.10


In [8]:
# Create a dictionary to map artist names to their order in artist_summary
artist_order = {artist: i for i, artist in enumerate(artist_summary['artist'])}

# Define a function to get the minimum order of artists in a track
def get_min_artist_order(row):
  artist_orders = [artist_order.get(row[artist], float('inf')) for artist in artist_columns if row[artist] in artist_order]
  return min(artist_orders) if artist_orders else float('inf')

# Apply the function to df_tracks to get the order for each track
df_tracks['artist_order'] = df_tracks.apply(get_min_artist_order, axis=1)

# Sort df_tracks by the artist_order
df_tracks_sorted = df_tracks.sort_values(by=['artist_order', 'album']).drop(columns='artist_order')

df_tracks_sorted.head()

,id,title,album,duration,release_date,rank,bpm,gain,artist_1,artist_2,...,in_CUD,in_ONE,in_ADD,in_RCN,in_SGM,in_VCL,in_LRB,in_FAV,in_JFA,in_EFA
120,2383115925,Living My Life,5am,225,2023-08-30,73926,0.0,-7.6,milet,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,2383115935,Noel In July,5am,286,2023-08-30,88865,0.0,-9.8,milet,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,2383115985,Hey Song,5am,211,2023-08-30,114814,0.0,-8.5,milet,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN
302,2383115995,Flare,5am,234,2023-08-30,51443,0.0,-7.9,milet,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,2383116025,HELL CLUB,5am,230,2023-08-30,72351,0.0,-8.1,milet,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
